In [ ]:
from importing import *

import json
import time
import pandas as pd
import numpy as np
from numpy.lib import recfunctions
from argparse import ArgumentParser
from TravelAndMutate.paramsmanager import Params
from TravelAndMutate.randominterface import NumpyRandomGenerator
from TravelAndMutate.system import SystemMutations as System
from TravelAndMutate.eventrecorder import EventRecorder
import TravelAndMutate.datamanager as datman

In [ ]:
def main(working_dir, filename, seed, suppress_output=False):

	if not working_dir[-1] == "/":
		working_dir = working_dir + "/"

	with open(working_dir+"params.json") as paramfile:
		params_dict = json.load(paramfile)

	random_engine = NumpyRandomGenerator(seed)
	
	params = Params(params_dict, random_engine.rng).__dict__
	patch_params = pd.DataFrame()
	patch_params["N"] = params["Ns"].astype("u4")
	patch_params["beta"] = params["betas"]
	patch_params["epsilon"] = params["epsilons"]
	patch_params["mu"] = params["mus"]
	patch_params["I0"] = params["I0"].astype("u4")

	system = System(random_engine.cpprng, params["commuting"], patch_params.to_records(index=False), params["gamma_trick"])
	system.setMutationRate(params["mutation_rate"])
	system.seedEpidemic()
	if not suppress_output:
		system.setVerbosity()

	starttime = time.time()
	system.spreadForTime(params["t_max"])
	simulationtime = time.time() - starttime

	starttime = time.time()
	storingtime = time.time() - starttime

	if not suppress_output:
		print(f"Time elapsed simulating: {round(simulationtime, 2)} s")
		print(f"Time elapsed storing data: {round(storingtime, 2)} s")

	return system

In [ ]:
working_dir = "data/"
filename = "periods"
seed = 4
suppress_output = False

system = main(working_dir, filename, seed, suppress_output)

In [ ]:
df = pd.DataFrame.from_records(system.getEventRecorder().read())
df["agent"] = (df["ID"] + df["loc"]*10**(np.ceil(np.log10(df["ID"].max())))).astype(int)
#df.drop(columns=["loc","ID"], inplace=True)
#df.set_index("agent", inplace=True)

In [ ]:
df

In [ ]:
_,what = np.unique(df["agent"], return_counts=True)

In [ ]:
np.unique(what, return_counts=True)

In [ ]:
agents = df["agent"].unique()
tlat = np.empty_like(agents)
tinf = np.empty_like(agents)
print("[")
for i,ag in enumerate(agents):
	sliced = df.loc[df["agent"]==ag]
	if sliced.shape[0] < 3:
		continue
	tlat[i] = np.diff(sliced["t"].loc[sliced["newstate"]!=82])[0]
	tinf[i] = np.diff(sliced["t"].loc[sliced["newstate"]!=69])[0]
	if tlat[i]>1000 or tinf[i]>1000:
		break
		print(ag, end=",")
	if i > 1000:
		break
tlat = tlat[:i+1]
tinf = tinf[:i+1]
print("]")

In [ ]:
import matplotlib.pyplot as plt

bins = np.linspace(0,100,101)
plt.hist(tlat, bins=bins, alpha=0.5)
plt.hist(tinf, bins=bins, alpha=0.5)
plt.axvline(tlat.mean(), linestyle="--")
plt.axvline(tinf.mean(), linestyle="--")
plt.axvline(1/0.1, linestyle=":", color="r")
plt.axvline(1/0.02, linestyle=":", color="r")
#plt.xscale("log")
#plt.yscale("log")
plt.tight_layout()